In [1]:
%bazel "@PythonKit//:PythonKit"
%bazel "@SwiftCrypto//:Crypto"
%bazel "//mujoco:jupyter"

Installing packages:
|-@PythonKit//:PythonKit
|-@SwiftCrypto//:Crypto
|-//mujoco:jupyter
Working in: /home/liu/Dropbox/workspace/sim2real/.ibzlnb/RPMas
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/mujoco/MuJoCoJupyterExtensions.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/external/PythonKit/PythonKit.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/external/SwiftCrypto/Crypto.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/external/SwiftNIO/NIO.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_liu/882e57e789e5af1ba36860a2b9ed2249/execroot/__main__/bazel-out/k8-opt/bin/external/SwiftNIO/NIOConcurrencyHelpers.swiftmodule
swiftmodule: /home/liu/.cache/bazel/_bazel_

In [6]:
import PythonKit

In [7]:
import MuJoCo
import MuJoCoJupyterExtensions

In [8]:
let simulate = Simulate(width: 960, height: 540, title: "simulate")
var m = try MjModel(fromXMLPath: "/home/liu/workspace/s4nnc/gym/assets/ant.xml")
var d = m.makeData()
simulate.use(model: m, data: &d)

In [9]:
let httpRenderServer = HTTPRenderServer(simulate, maxWidth: 960, maxHeight: 540, canResize: false)
let channel = try httpRenderServer.bind(host: "0.0.0.0", port: 12345).wait()

In [10]:
simulate.makeContext(hidden: true)

In [ ]:
JupyterDisplay.display(html: httpRenderServer.html)
JupyterDisplay.flush()
simulate.run()